In [30]:
import pandas as pd

In [31]:
df = pd.read_csv("en openfoodfacts org products.csv")

df.head()

,code,product_name,quantity,packaging_en,brands_en,categories_en,origins_en,labels_en,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g
0,301762042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finland,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN
1,2045652893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finland,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN
2,3017620422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finland,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN
3,490000030464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finland,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN
4,11702334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finland,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df.tail(10)

,code,product_name,quantity,packaging_en,brands_en,categories_en,origins_en,labels_en,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g
13209,6408430062195,Karviais-boysenmarjakeitto+kuitu sokeroimaton,1l,"Cardboard,Paperboard",Valio,"Beverages,Artificially sweetened beverages",NaN,NaN,Finland,unknown,Beverages,Artificially sweetened beverages,"Beverages,Artificially sweetened beverages",NaN,unknown,Artificially sweetened beverages,NaN,NaN
13210,6408430407217,Profeel proteiinipirtelö appelsiinikaakao,NaN,"Tetra Pak,Tetra Top",Valio,"Beverages,Dairies,Dairy drinks,Flavoured milks...",NaN,NaN,Finland,not-applicable,Beverages,Artificially sweetened beverages,"Beverages,Artificially sweetened beverages",NaN,unknown,Bodybuilding supplements,NaN,NaN
13211,6408430408184,Profeel proteiinipirtelö,NaN,"Tetra Pak,Tetra Top",Valio,"Beverages,Dairies,Dairy drinks,Flavoured milks...",NaN,NaN,Finland,unknown,Beverages,Artificially sweetened beverages,"Beverages,Artificially sweetened beverages",NaN,unknown,Protein shakes,NaN,NaN
13212,6411402164902,Reilu,500 g,Plastic,Oululainen,"Plant-based foods and beverages,Plant-based fo...",NaN,NaN,Finland,e,Cereals and potatoes,Bread,"Cereals and potatoes,Bread",NaN,unknown,Breads,NaN,NaN
13213,6411402578525,Imatran riisipiirakka,6 kpl,fi:kartonki-muovi,Fazer,"Meals,Pizzas pies and quiches,Quiches",NaN,NaN,Finland,c,Composite foods,Pizza pies and quiches,"Composite foods,Pizza pies and quiches",NaN,unknown,Quiches,NaN,NaN
13214,6413466015305,Rukiinen riisipiirakka,6 kpl,fi:paperi,Fazer,fi:riisipiirakat,NaN,NaN,Finland,c,unknown,unknown,NaN,NaN,unknown,fi:riisipiirakat,NaN,NaN
13215,6420256010860,NaN,120 g,"Plastic,Other-plastics",Cloetta,"Snacks,Sweet snacks,Cocoa and its products,Con...",NaN,NaN,Finland,e,Sugary snacks,Sweets,"Sugary snacks,Sweets",NaN,unknown,Bonbons,NaN,NaN
13216,6430053410015,Naan Tandoor - Leipä,250 g,fi:muovipussi,Kirkuk-leipomo,"Plant-based foods and beverages,Plant-based fo...",NaN,NaN,Finland,unknown,Cereals and potatoes,Bread,"Cereals and potatoes,Bread",NaN,unknown,Naans,NaN,NaN
13217,7310532151976,Chili nuts,140g,Plastic,estrella,NaN,NaN,NaN,Finland,unknown,unknown,unknown,NaN,NaN,unknown,NaN,NaN,NaN
13218,7322550037514,Suurustettu kana,60 g,fi:pahvi,Lammin-kuppi,"Meals,Soups,Reheatable soups,fi:kei",NaN,NaN,Finland,c,Composite foods,One-dish meals,"Composite foods,One-dish meals",NaN,unknown,fi:kei,NaN,NaN


In [33]:
df["pnns_groups_2"].value_counts()

pnns_groups_2
unknown                             8484
Sweets                               510
Biscuits and cakes                   382
Milk and yogurt                      363
One-dish meals                       283
Dairy desserts                       279
Bread                                251
Cheese                               194
Dressings and sauces                 189
Breakfast cereals                    185
Chocolate products                   176
Meat                                 169
Cereals                              162
Sweetened beverages                  148
Ice cream                            131
Processed meat                       128
Plant-based milk substitutes         113
Vegetables                            92
Artificially sweetened beverages      83
Alcoholic beverages                   80
Unsweetened beverages                 69
Fats                                  68
Fish and seafood                      65
Appetizers                            64
Sa

In [34]:
df.isnull().sum()

code                                           0
product_name                                 761
quantity                                    8634
packaging_en                               11160
brands_en                                   1380
categories_en                               7813
origins_en                                 12184
labels_en                                   9980
countries_en                                   0
nutriscore_grade                              87
pnns_groups_1                                 85
pnns_groups_2                                 85
food_groups_en                              8547
environmental_score_score                  12236
environmental_score_grade                   9638
main_category_en                            7813
carbon-footprint_100g                      13217
carbon-footprint-from-meat-or-fish_100g    13213
dtype: int64

In [35]:
df.shape

(13219, 18)

In [36]:
df.drop(df[df['pnns_groups_1'] == "unknown"].index, inplace=True)


In [37]:
df.shape

(4735, 18)